Cài đặt các thư viện cần thiết:


In [1]:
cp -r /kaggle/input/check-point-t5/flan-t5-text2sql-lora /kaggle/working/


In [2]:
!pip install -q wandb

In [3]:
!pip install -q transformers datasets accelerate peft bitsandbytes torch tensorboard huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 21.7 MB/s eta 0:00:00:00:0100:01


In [4]:
pip install -U q transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 66.5 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from huggingface_hub import login

login("key")


In [6]:
import torch
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

# Kiểm tra xem có GPU không
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load mô hình và tokenizer
BASE_MODEL_NAME = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(BASE_MODEL_NAME, legacy=False)
base_model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME).to(device)




tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:

# Load dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql")

README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [8]:

# Chuẩn bị dữ liệu huấn luyện
def preprocess_function(example):
    input_texts = [f"Convert to SQL: {p} Context: {c}" for p, c in zip(example['sql_prompt'], example['sql_context'])]
    target_texts = example['sql']

    inputs = tokenizer(
        input_texts,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="np"  # Đảm bảo đầu ra đồng nhất
    )

    targets = tokenizer(
        target_texts,
        max_length=256,
        truncation=True,
        padding="max_length",
        return_tensors="np"
    )

    return {
        "input_ids": inputs["input_ids"].tolist(),
        "attention_mask": inputs["attention_mask"].tolist(),
        "labels": targets["input_ids"].tolist()
    }

# Áp dụng tiền xử lý
train_dataset = dataset["train"].map(preprocess_function, batched=True)
test_dataset = dataset["test"].map(preprocess_function, batched=True)


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [9]:
import torch
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# Cấu hình LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "k", "v", "o"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

# Bọc mô hình với LoRA
base_model_lora_wrapper = get_peft_model(base_model, lora_config)

In [10]:
# Số lượng tham số ban đầu của mô hình
total_params = sum(p.numel() for p in base_model_lora_wrapper.parameters())
trainable_params = sum(p.numel() for p in base_model_lora_wrapper.parameters() if p.requires_grad)

# In kết quả
print(f"Tổng số tham số: {total_params:,}")
print(f"Số tham số được huấn luyện: {trainable_params:,}")
print(f"Tỷ lệ tham số huấn luyện: {trainable_params / total_params:.2%}")


Tổng số tham số: 792,587,264
Số tham số được huấn luyện: 9,437,184
Tỷ lệ tham số huấn luyện: 1.19%


In [ ]:
import wandb

wandb.login(key="key")
wandb.init(project="flan-t5-text2sql-lora", name="train_kaggle", resume="allow")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ugw3005 (ugw3005-hust). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:

# Cấu hình huấn luyện cho Kaggle
OUTPUT_DIR = "./flan-t5-text2sql-lora"
EVAL_STRATEGY = "no"
SAVE_STRATEGY = "steps"
SAVE_STEPS = 2500
TRAIN_BATCH_SIZE = 2  # Giảm batch size để tránh lỗi bộ nhớ
EVAL_BATCH_SIZE = 2
LEARNING_RATE = 3e-5
WEIGHT_DECAY = 0.01
NUM_EPOCHS = 3
LOGGING_DIR = "./logs"
LOGGING_STEPS = 50
SAVE_LIMIT = 3  # Giảm số lượng checkpoints để tiết kiệm dung lượng
BF16_ENABLED = torch.cuda.is_bf16_supported()  # Chỉ dùng bf16 nếu GPU hỗ trợ
PUSH_TO_HUB = True  
HUB_MODEL_ID = "ngocnamk3er/flan-t5-text2sql-lora"
HUB_STRATEGY = "every_save"
GRAD_ACCUM_STEPS = 1
DATALOADER_WORKERS = 2  # Giảm số lượng workers để tránh quá tải
OPTIMIZER = "adamw_torch"  # Dùng AdamW mặc định thay vì 8-bit

In [13]:


# Cấu hình training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy=EVAL_STRATEGY,
    save_strategy=SAVE_STRATEGY,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    num_train_epochs=NUM_EPOCHS,
    # logging_dir=LOGGING_DIR,
    logging_steps=LOGGING_STEPS,
    save_total_limit=SAVE_LIMIT,
    bf16=BF16_ENABLED,
    push_to_hub=PUSH_TO_HUB,
    report_to="wandb",
    hub_model_id=HUB_MODEL_ID,
    hub_strategy=HUB_STRATEGY,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    dataloader_num_workers=DATALOADER_WORKERS,
    optim=OPTIMIZER,
    save_steps=SAVE_STEPS,
    max_steps=35000,
    resume_from_checkpoint=True
)


In [14]:

# Trainer
trainer = Trainer(
    model=base_model_lora_wrapper,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Huấn luyện mô hình
# trainer.train()
trainer.train(resume_from_checkpoint = True)

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3423: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We rec

Step,Training Loss
25050,0.100200
25100,0.101800
25150,0.097000
25200,0.092200
25250,0.096700
25300,0.096000
25350,0.092300
25400,0.100400
25450,0.107000
25500,0.081400


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=35000, training_loss=0.027013368340900967, metrics={'train_runtime': 19637.1088, 'train_samples_per_second': 7.129, 'train_steps_per_second': 1.782, 'total_flos': 3.2672625721344e+17, 'train_loss': 0.027013368340900967, 'epoch': 1.4})

In [15]:
!zip -r flan-t5-text2sql-lora.zip ./flan-t5-text2sql-lora

  adding: flan-t5-text2sql-lora/ (stored 0%)
  adding: flan-t5-text2sql-lora/training_args.bin (deflated 51%)
  adding: flan-t5-text2sql-lora/README.md (deflated 46%)
  adding: flan-t5-text2sql-lora/adapter_model.safetensors (deflated 7%)
  adding: flan-t5-text2sql-lora/checkpoint-35000/ (stored 0%)
  adding: flan-t5-text2sql-lora/checkpoint-35000/training_args.bin (deflated 51%)
  adding: flan-t5-text2sql-lora/checkpoint-35000/trainer_state.json (deflated 84%)
  adding: flan-t5-text2sql-lora/checkpoint-35000/README.md (deflated 66%)
  adding: flan-t5-text2sql-lora/checkpoint-35000/adapter_model.safetensors (deflated 7%)
  adding: flan-t5-text2sql-lora/checkpoint-35000/optimizer.pt (deflated 8%)
  adding: flan-t5-text2sql-lora/checkpoint-35000/rng_state.pth (deflated 25%)
  adding: flan-t5-text2sql-lora/checkpoint-35000/adapter_config.json (deflated 54%)
  adding: flan-t5-text2sql-lora/checkpoint-35000/scheduler.pt (deflated 57%)
  adding: flan-t5-text2sql-lora/checkpoint-32500/ (store

In [16]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/ngocnamk3er/flan-t5-text2sql-lora/commit/8f58f0ff9821de03d66bcd2ef410695b25e04762', commit_message='End of training', commit_description='', oid='8f58f0ff9821de03d66bcd2ef410695b25e04762', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ngocnamk3er/flan-t5-text2sql-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='ngocnamk3er/flan-t5-text2sql-lora'), pr_revision=None, pr_num=None)